# Basic Widget Manufacturer Model

## Overview
- **What this notebook does:** Walks through the fundamental financial model that underpins every simulation in this framework -- balance sheet dynamics, revenue generation, insurance claims, and multi-year growth.
- **Prerequisites:** [getting-started/01_setup_verification.ipynb](01_setup_verification.ipynb)
- **Estimated runtime:** < 1 minute
- **Audience:** [Practitioner]

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/getting-started'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

from ergodic_insurance import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# Reproducibility
np.random.seed(42)

## Configuration

All parameters are defined here so the notebook is easy to re-run with different assumptions.

In [ ]:
INITIAL_ASSETS = 10_000_000       # $10M
ASSET_TURNOVER = 1.0              # $1 revenue per $1 assets
OPERATING_MARGIN = 0.08           # 8% EBIT margin
TAX_RATE = 0.25                   # 25% corporate tax
RETENTION_RATIO = 1.0             # 100% earnings retained

DEDUCTIBLE = 1_000_000            # $1M insurance deductible
INSURANCE_LIMIT = 10_000_000      # $10M insurance limit
SIMULATION_YEARS = 10

# Claims that occur during the simulation
CLAIM_SCHEDULE = {
    2: 5_000_000,    # $5M claim in year 2
    6: 12_000_000,   # $12M claim in year 6
}

## 1. Initialize the Manufacturer

The `WidgetManufacturer` models a generic manufacturing business with assets, revenue, and equity. Key financial ratios drive the simulation:
- **Asset Turnover:** Revenue generated per dollar of assets
- **Operating Margin:** Profit before interest and taxes as a fraction of revenue
- **Retention Ratio:** Fraction of earnings retained (vs. distributed as dividends)

In [ ]:
config = ManufacturerConfig(
    initial_assets=INITIAL_ASSETS,
    asset_turnover_ratio=ASSET_TURNOVER,
    base_operating_margin=OPERATING_MARGIN,
    tax_rate=TAX_RATE,
    retention_ratio=RETENTION_RATIO,
)

manufacturer = WidgetManufacturer(config)

print("Initial Balance Sheet")
print("=" * 40)
print(f"Assets:            ${float(manufacturer.total_assets):,.0f}")
print(f"Restricted Assets: ${float(manufacturer.restricted_assets):,.0f}")
print(f"Available Assets:  ${float(manufacturer.available_assets):,.0f}")
print(f"Equity:            ${float(manufacturer.equity):,.0f}")
print(f"Collateral (LoC):  ${float(manufacturer.collateral):,.0f}")

## 2. Single-Year Operations

Before running a multi-year simulation, let's trace the financial flows for one year to understand how revenue becomes net income.

In [ ]:
revenue = manufacturer.calculate_revenue()
operating_income = manufacturer.calculate_operating_income(revenue)
net_income = manufacturer.calculate_net_income(operating_income, collateral_costs=0)

print("Annual Financial Performance")
print("=" * 40)
print(f"Revenue:           ${float(revenue):,.0f}")
print(f"Operating Income:  ${float(operating_income):,.0f} ({OPERATING_MARGIN:.0%} margin)")
print(f"Net Income:        ${float(net_income):,.0f} (after {TAX_RATE:.0%} tax)")
print(f"Return on Equity:  {float(net_income) / float(manufacturer.equity):.1%}")
print(f"Return on Assets:  {float(net_income) / float(manufacturer.total_assets):.1%}")

## 3. Insurance Claim Processing

Insurance claims are split between the company and the insurer based on two parameters:
- **Deductible:** Amount the company pays before insurance kicks in.
- **Limit:** Maximum amount the insurer covers above the deductible.

In [ ]:
test_claims = [500_000, 3_000_000, 15_000_000]

print(f"Insurance Parameters: Deductible=${DEDUCTIBLE:,.0f}, Limit=${INSURANCE_LIMIT:,.0f}")
print("=" * 60)

for claim_amount in test_claims:
    if claim_amount <= DEDUCTIBLE:
        company_pays = claim_amount
        insurance_pays = 0
    else:
        company_pays = DEDUCTIBLE
        insurance_pays = min(claim_amount - DEDUCTIBLE, INSURANCE_LIMIT)
        if claim_amount > DEDUCTIBLE + INSURANCE_LIMIT:
            company_pays += claim_amount - DEDUCTIBLE - INSURANCE_LIMIT

    print(f"\nClaim: ${claim_amount:,.0f}")
    print(f"  Company pays:   ${company_pays:,.0f}")
    print(f"  Insurance pays: ${insurance_pays:,.0f}")

## 4. Multi-Year Simulation

Simulate 10 years of operations with scheduled claims in years 2 and 6 to see how the business evolves.

In [ ]:
manufacturer.reset()

years, assets, equity, collateral, roe = [0], [float(manufacturer.total_assets)], [float(manufacturer.equity)], [float(manufacturer.collateral)], [0.0]

for year in range(1, SIMULATION_YEARS + 1):
    if year in CLAIM_SCHEDULE:
        claim_amount = CLAIM_SCHEDULE[year]
        company_payment, insurance_payment = manufacturer.process_insurance_claim(
            claim_amount, DEDUCTIBLE, INSURANCE_LIMIT
        )
        print(f"Year {year}: Claim ${claim_amount:,.0f} "
              f"(Company: ${float(company_payment):,.0f}, Insurance: ${float(insurance_payment):,.0f})")

    metrics = manufacturer.step(letter_of_credit_rate=0.015, growth_rate=0.03)

    years.append(year)
    assets.append(float(metrics['assets']))
    equity.append(float(metrics['equity']))
    collateral.append(float(metrics['collateral']))
    roe.append(float(metrics['roe']))

results_df = pd.DataFrame({
    'Year': years, 'Assets': assets, 'Equity': equity,
    'Collateral': collateral, 'ROE': roe,
})

## 5. Visualize Financial Evolution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Widget Manufacturer: 10-Year Financial Evolution', fontsize=16, y=1.02)

# Balance sheet
ax = axes[0, 0]
ax.plot(results_df['Year'], results_df['Assets'] / 1e6, 'b-', lw=2, label='Assets')
ax.plot(results_df['Year'], results_df['Equity'] / 1e6, 'g-', lw=2, label='Equity')
ax.set_xlabel('Year'); ax.set_ylabel('Value ($M)'); ax.set_title('Balance Sheet Evolution')
ax.legend(); ax.grid(True, alpha=0.3)

# Collateral
ax = axes[0, 1]
ax.fill_between(results_df['Year'], 0, results_df['Collateral'] / 1e6, color='orange', alpha=0.6)
for yr, amt in CLAIM_SCHEDULE.items():
    ax.axvline(x=yr, color='red', ls='--', alpha=0.5)
ax.set_xlabel('Year'); ax.set_ylabel('Collateral ($M)'); ax.set_title('Letter of Credit Collateral')
ax.grid(True, alpha=0.3)

# ROE
ax = axes[1, 0]
ax.bar(results_df['Year'][1:], results_df['ROE'][1:] * 100, color='darkgreen', alpha=0.7)
ax.axhline(y=6, color='red', ls='--', alpha=0.5, label='Target ROE (6%)')
ax.set_xlabel('Year'); ax.set_ylabel('ROE (%)'); ax.set_title('Return on Equity')
ax.legend(); ax.grid(True, alpha=0.3)

# Asset growth rate
ax = axes[1, 1]
asset_growth = results_df['Assets'].pct_change()[1:] * 100
ax.plot(results_df['Year'][1:], asset_growth, 'b-', lw=2, marker='o')
ax.axhline(y=0, color='black', ls='-', alpha=0.3)
ax.set_xlabel('Year'); ax.set_ylabel('Growth Rate (%)'); ax.set_title('Asset Growth Rate')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Financial Summary

In [ ]:
print(f"Financial Performance Summary ({SIMULATION_YEARS} Years)")
print("=" * 50)
print(f"Initial Assets:  ${results_df['Assets'].iloc[0]:,.0f}")
print(f"Final Assets:    ${results_df['Assets'].iloc[-1]:,.0f}")
print(f"Asset CAGR:      {((results_df['Assets'].iloc[-1] / results_df['Assets'].iloc[0]) ** (1 / SIMULATION_YEARS) - 1) * 100:.1f}%")
print()
print(f"Initial Equity:  ${results_df['Equity'].iloc[0]:,.0f}")
print(f"Final Equity:    ${results_df['Equity'].iloc[-1]:,.0f}")
print(f"Equity CAGR:     {((results_df['Equity'].iloc[-1] / results_df['Equity'].iloc[0]) ** (1 / SIMULATION_YEARS) - 1) * 100:.1f}%")
print()
print(f"Average ROE:     {results_df['ROE'][1:].mean() * 100:.1f}%")
print(f"Total Claims:    ${sum(CLAIM_SCHEDULE.values()):,.0f}")

## 7. Deductible Sensitivity

How do different deductible levels affect long-term performance? Lower deductibles transfer more risk to the insurer but cost more in premiums.

In [ ]:
deductibles = [0, 500_000, 1_000_000, 2_000_000, 5_000_000]
deductible_results = []

for ded in deductibles:
    manufacturer.reset()
    for year in range(1, SIMULATION_YEARS + 1):
        if year in CLAIM_SCHEDULE:
            manufacturer.process_insurance_claim(CLAIM_SCHEDULE[year], ded, INSURANCE_LIMIT)
        manufacturer.step(letter_of_credit_rate=0.015)

    deductible_results.append({
        'Deductible': ded,
        'Final_Equity': float(manufacturer.equity),
        'Avg_ROE': np.mean([float(m['roe']) for m in manufacturer.metrics_history]) * 100,
    })

ded_df = pd.DataFrame(deductible_results)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(ded_df['Deductible'] / 1e6, ded_df['Final_Equity'] / 1e6, 'b-o', lw=2, ms=8)
ax1.set_xlabel('Deductible ($M)'); ax1.set_ylabel('Final Equity ($M)')
ax1.set_title('Impact of Deductible on Final Equity'); ax1.grid(True, alpha=0.3)

ax2.plot(ded_df['Deductible'] / 1e6, ded_df['Avg_ROE'], 'g-s', lw=2, ms=8)
ax2.set_xlabel('Deductible ($M)'); ax2.set_ylabel('Average ROE (%)')
ax2.set_title('Impact of Deductible on Average ROE'); ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDeductible Sensitivity Analysis:")
print(ded_df.to_string(index=False, float_format='%.1f'))

## Key Takeaways

- The `WidgetManufacturer` provides a simple but complete financial model for simulation.
- Insurance claims split between company-retained losses and insurer-covered losses based on deductible and limit.
- Multi-year simulations reveal how claims, collateral, and growth interact over time.
- Deductible selection is a risk-return tradeoff: lower deductibles reduce volatility but increase insurance cost.

## Next Steps

- **Understand loss modeling:** [core/01_loss_distributions.ipynb](../core/01_loss_distributions.ipynb)
- **See the ergodic advantage:** [core/03_ergodic_advantage.ipynb](../core/03_ergodic_advantage.ipynb)
- **Run large-scale simulations:** [core/04_monte_carlo_simulation.ipynb](../core/04_monte_carlo_simulation.ipynb)
- **Analyze growth dynamics:** [core/07_growth_dynamics.ipynb](../core/07_growth_dynamics.ipynb)